In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/temp_data/cv_competition_data.zip' -d data/


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

import time
import os
import shutil
from PIL import Image
from tqdm import tqdm

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, mode, transform=None):
        """
        Args:
            root_dir (string): 项目根目录。
            mode (string): 模式，包含train, validate, test
            transform (callable, optional): 可选的转换操作。
        """
        if transform is None:
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
            ])
            
        self.root_dir = root_dir
        self.transform = transform
        
        self.annotation_path = os.path.join(root_dir, f'data/annotations/{"val" if mode == "test" else "train"}.txt')
        self.img_path_dir = os.path.join(root_dir, f'data/{"val" if mode == "test" else "train"}set')
        
        self.samples = self.get_annotations(mode, self.annotation_path)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_name, label = self.samples[idx]
        img_path = os.path.join(self.img_path_dir, img_name).replace("*", "_")
        
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)
            
        return (img_name, image), label

    def get_annotations(self, mode, annotation_path):
        image_labels = []
        with open(annotation_path, 'r') as file:
            for line in file:
                img_name, label = line.strip().split()
                image_labels.append((img_name, int(label)))
                
        count = len(image_labels)
        # count = 100
        if mode == "validate":
            image_labels = image_labels[:count//10]
        elif mode == "train":
            image_labels = image_labels[count//10:count]
            
        return image_labels
    
def get_dataset(root_dir, mode):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    
    dataset = CustomDataset(root_dir, mode, transform)
    return dataset
    
    
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg,(1, 2, 0)))
    plt.show()

In [ ]:
# 定义转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 项目根目录
root_dir = 'D://fudan//2024Autumn//CV//competition//cv_competition'

# 创建数据集和数据加载器
dataset = CustomDataset(root_dir=root_dir, mode="train", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 遍历数据并打印图像和标签的形状
for (names, images), labels in dataloader:
    print(f'Batch of images shape: {images.shape}')  # 打印图像的形状
    print(f'Batch of image names: {names}')  # 打印标签
    print(f'Batch of labels: {labels}')  # 打印标签
    break  # 只打印一个批次的数据以进行测试

# get some random training images
dataiter = iter(dataloader)
(names, images), labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
# print labels
print('Labels:\n', '\n'.join('%20s: %3d;' % (names[j], labels[j]) for j in range(32)))

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # 输入通道为3 (RGB)，输出通道为32
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # 下采样一半
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), # 卷积层
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)                  # 再次下采样
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),                                         # 展平
            nn.Linear(64 * 56 * 56, 128),                        # 全连接层
            nn.ReLU(),
            nn.Linear(128, 1)                                    # 最终输出单个值（预测年龄）
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x
    
    
class MyResNetModel(nn.Module):
    def __init__(self):
        super(MyResNetModel, self).__init__()
        resnet = models.resnet50(pretrained=True)

        for param in resnet.parameters():
            param.requires_grad = False

        self.resnet = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(resnet.fc.in_features, 1)

    def forward(self, x):
        x = self.resnet(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x
    
    
class MyResNetModel2(nn.Module):
    def __init__(self):
        super(MyResNetModel2, self).__init__()
        resnet = models.resnet50(pretrained=True)

        for param in resnet.parameters():
            param.requires_grad = False
            
        for param in resnet.layer4.parameters():
            param.requires_grad = True

        self.resnet = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(resnet.fc.in_features, 1)

    def forward(self, x):
        x = self.resnet(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [ ]:
def train(device, train_loader, validate_loader, model, optimizer, criterion, max_epochs, save_path):
    """
    Train the model for regression with early stopping based on validation loss.

    Args:
        device (torch.device): The device to use for training (e.g., "cuda" or "cpu").
        train_loader (DataLoader): DataLoader for the training dataset.
        validate_loader (DataLoader): DataLoader for the validation dataset.
        model (nn.Module): The model to train.
        optimizer (torch.optim.Optimizer): Optimizer for model training.
        criterion (nn.Module): Loss function for regression (e.g., MSELoss).
        max_epochs (int): Maximum number of epochs to train.
        save_path (str): Directory to save the best model.
    """
    best_loss = float('inf')
    patience = 3  # Stop if no improvement for 3 consecutive epochs
    patience_counter = 0

    for epoch in range(max_epochs):
        model.train()
        running_loss = 0.0

        # Training loop
        for (names, images), labels in tqdm(train_loader, desc="Epoch_" + str(epoch) + " Train Processing:"):
            images, labels = images.to(device), labels.to(device).float()

            # Forward pass
            outputs = model(images).squeeze(1)  # Ensure outputs are 1D
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{max_epochs}], Train Loss: {avg_train_loss:.4f}")

        # Validation loop
        model.eval()
        validate_loss = 0.0

        with torch.no_grad():
            for (names, images), labels in validate_loader:
                images, labels = images.to(device), labels.to(device).float()

                outputs = model(images).squeeze(1)
                loss = criterion(outputs, labels)
                validate_loss += loss.item()

        avg_validate_loss = validate_loss / len(validate_loader)
        print(f"Validation Loss: {avg_validate_loss:.4f}")

        is_best = False
        # Check for improvement
        if avg_validate_loss < best_loss:
            print("Validation loss improved, saving model...")
            is_best = True
            best_loss = avg_validate_loss
            patience_counter = 0
        else:
            patience_counter += 1
        save_checkpoint(model.state_dict(), save_path, is_best, epoch)
        
        # Early stopping
        if patience_counter >= patience:
            print("Early stopping triggered. Best validation loss:", best_loss)
            break

    print("Training complete. Best validation loss:", best_loss)



def save_checkpoint(state, save_root, is_best, epoch):
    if not os.path.exists(save_root):
        os.makedirs(save_root)
    save_path = os.path.join(save_root, 'epoch_{}.pth.tar'.format(str(epoch)))
    torch.save(state, save_path)
    
    best_path = os.path.join(save_root, 'best_model.pth.tar'.format(str(epoch)))
    torch.save(state, best_path)

In [ ]:
print("Start...")
root_dir = '.'
batch_size = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyResNetModel2().to(device)

criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_dataset = get_dataset(root_dir, "train")
validate_dataset = get_dataset(root_dir, "validate")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, drop_last=False)
validate_loader = torch.utils.data.DataLoader(validate_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, drop_last=False)

train(
    device,
    train_loader,
    validate_loader,
    model,
    optimizer,
    criterion,
    max_epochs=50,
    save_path=os.path.join(root_dir, "save")
)